<a href="https://colab.research.google.com/github/NavidJavaheri/AI/blob/main/calory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xzf food-101.tar.gz

--2026-01-04 10:06:52--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2026-01-04 10:06:53--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz.1’

food-101.tar.gz.1   100%[===================>]   4.65G  22.1MB/s    in 4m 0s   

2026-01-04 10:10:53 (19.9 MB/s) - ‘food-101.tar.gz.1’ saved [4996278331/4996278331]



In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    'food-101/images',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    'food-101/images',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical'
)


Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(101, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.fit(
    train_gen,
    epochs=1,
    steps_per_epoch=100
)


100/100 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.4924 - loss: 2.0170


In [ ]:
calorie_dict = {
    'apple_pie': 237,
    'baby_back_ribs': 292,
    'baklava': 428,
    'beef_carpaccio': 126,
    'beef_tartare': 187,
    'cheesecake': 321,
    'chicken_curry': 168,
    'chicken_wings': 203,
    'chocolate_cake': 371,
    'french_fries': 312,
    'fried_rice': 163,
    'hamburger': 295,
    'hot_dog': 290,
    'ice_cream': 207,
    'pizza': 266,
    'ramen': 436,
    'samosa': 262,
    'spaghetti_bolognese': 158,
    'sushi': 130,
    'tiramisu': 283
}


In [ ]:
from tensorflow.keras.preprocessing import image

def predict_calories(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)
    class_index = np.argmax(pred)
    class_name = list(train_gen.class_indices.keys())[class_index]

    calories = calorie_dict.get(class_name, "Unknown")

    print("Food:", class_name)
    print("Estimated Calories (per 100g):", calories)


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving bw.jpeg to bw.jpeg


In [ ]:
predict_calories("bw.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Food: chicken_wings
Estimated Calories (per 100g): 203
